<a href="https://colab.research.google.com/github/MoAbeds/news-summary-api/blob/main/news_summary_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
!pip install transformers --quiet
!pip install sentencepiece --quiet
!pip install newsapi-python --quiet
!pip install fastapi
!pip install "uvicorn[standard]"



In [61]:
import torch
import requests
from transformers import pipeline
from bs4 import BeautifulSoup
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import pandas as pd
from newsapi import NewsApiClient
from typing import Optional
from fastapi import FastAPI


In [62]:
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

In [79]:
NEWS_API = 'dd9e2fb9c5484b00983d86dd515384b2'
news_client = NewsApiClient(api_key=NEWS_API)

Search_input = 'apple'
top_headlines = news_client.get_top_headlines(q=Search_input,
                                          country='us',
                                          language='en', )

In [ ]:
top_headlines

In [75]:
NewsArr=[]
for atr in top_headlines['articles']:
  title = atr['title']
  content = atr['content']
  
  if content is not None:
    tokens = tokenizer(content, truncation=True, padding="longest", return_tensors="pt")
    summary = model.generate(**tokens)
    sum = tokenizer.decode(summary[0])
    Blogdic = { 'title': title,'content':content,'summary': sum}
    NewsArr.append(Blogdic)

  else:
    pass

  



In [ ]:
#create api
app = FastAPI()


@app.get("/")
def read_root():
    return {"Hello": "World"}


@app.get("/news/{item_id}")
def read_item(search_input: int, q: Optional[str] = None):
  Search_input = search_input
  top_headlines = news_client.get_top_headlines(q=Search_input,
                                          country='us',
                                          language='en', )
  NewsArr=[]
  for atr in top_headlines['articles']:
    title = atr['title']
    content = atr['content']
    
    if content is not None:
      tokens = tokenizer(content, truncation=True, padding="longest", return_tensors="pt")
      summary = model.generate(**tokens)
      sum = tokenizer.decode(summary[0])
      Blogdic = { 'title': title,'content':content,'summary': sum}
      NewsArr.append(Blogdic)

    else:
      pass

  return NewsArr

  
